<img src="imgs/IISc_Master_Seal_Black_Transparent.png" height="120px" width="120px" align="right" />

<img src="imgs/logoGESS.jpg" height="120px" width="120px" align="right" />

<font face="Calibri">
<font size="7"> <b> PySHBundle Tutorials </b> </font>
<br> 
<font size="5"> <b> Tutorial 1: Introduction to Spherical Harmonics using PySHBundle<font color='rgba(200,0,0,0.2)'>  </font> </b> </font>
<br> <br>
    
<font size="3"> <b> by: Abhishek Mhamane, IIT Kanpur</b> 
<font size="2">  <br>
<font> <b>Date: </b> July 1, 2023 </font>
</font>


In [ ]:
import pandas as pd
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pyshbundle

In [ ]:
from pyshbundle.shutils import PLM
from pyshbundle.viz_utils import ylm, ylm_plot

# What are Spherical Harmonics?

Solid and Surface spherical harmonics
1. Harmonics - solutions of Laplace equation
2. Spherical - they are in spherical coordinates ($\theta$, $\lambda$, <em>r</em>)
3. Solid - cause they are 3D functions 
4. Surface - when we remove the radial part (or consider a unit sphere)

# Representing Gravitational potential as spherical harmonic expansion

$$
V(r, \theta, \lambda) = \frac{GM}{R} \sum_{l=0}^{\infty} \left(\frac{R}{r} \right)^{l+1} \sum_{m=0}^{l} P_{lm}(\cos{\theta}) (C_{lm}\cos{m\lambda} + S_{lm}\sin{m\lambda})
$$

Where, <br>
- <em>l</em> is the spherical harmonic **degree** <br>
- <em>m </em> is the spherical harmonic **order**

## Associated Legendre Polynomials


In [ ]:
help(PLM)

In [ ]:
#
l = np.zeros((1,1))
l[0] = 7

l2 = np.zeros((1,1))
l2[0] = 7
n = [0, 7, 14, 20]
colatitude = np.deg2rad(np.arange(0, 180.5, 0.5))
lon = np.deg2rad(np.arange(-180, 180.5, 0.5))

In [ ]:
print(l)

In [ ]:
# computing polynomial 
plm_7_0 = PLM(l=l, m=0, thetaRAD=colatitude, nargin=1, nargout=1)
plm_7_4 = PLM(l=l2, m=4, thetaRAD=colatitude, nargin=1, nargout=1)
plm_7_7 = PLM(l=l2, m=7, thetaRAD=colatitude, nargin=1, nargout=1)

In [ ]:
plm_7_0.shape

In [ ]:
# Plotting the plm 
fig, ax = plt.subplots(figsize=(8, 5))
plt.plot(np.rad2deg(colatitude), plm_7_0[:, 0, 0], label="PLM(7, 0)")
plt.plot(np.rad2deg(colatitude), plm_7_4[:, 0, 0], label="PLM(7, 4)")
plt.plot(np.rad2deg(colatitude), plm_7_7[:, 0, 0], label="PLM(7, 7)")

plt.hlines(y=0, xmin=0, xmax=180, linestyles='--', colors='k')
plt.xlim([0, 180])
plt.legend()

plt.title("Visualizing the PLM function -> number of zeros = degree - order")
plt.xlabel("Co-latitude")
plt.ylabel("plm(l, m)")

plt.show()

## Check Orthogonality of Spherical Harmonic Coefficients

Refering to Stack Overflow post - https://stackoverflow.com/questions/39537794/orthogonality-issue-in-scipys-legendre-polynomials

Great GitHub Repo by @markmbaum for 2-D Orthogonal Spherical Harmonic. Find the docs for `orthopoly` - https://markmbaum.github.io/orthopoly/

## Visualizing the orthogonality of Legendre Polynomial

In [ ]:
# the plm implementation using a co-latitude vector (geodetic use-case)
l1 = np.zeros((1,1))
l1[0] = 2

l2 = np.zeros((1,1))
l2[0] = 4


colatitude = np.deg2rad(np.arange(0, 180.5, 0.5))

plm_2_0 = PLM(l=l1, m=0, thetaRAD=colatitude, nargin=1, nargout=1)
plm_4_0 = PLM(l=l2, m=0, thetaRAD=colatitude, nargin=1, nargout=1)

product_2_4 = plm_2_0*plm_4_0

# Plotting the plm 
fig, ax = plt.subplots(figsize=(9, 5))
plt.plot(np.rad2deg(colatitude), plm_2_0[:, 0, 0], label="PLM(2, 0)")
plt.plot(np.rad2deg(colatitude), plm_4_0[:, 0, 0], label="PLM(4, 0)")

plt.plot(np.rad2deg(colatitude), product_2_4[:, 0, 0], label="PLM(2, 0)*PLM(4, 0)")

plt.fill_between(np.rad2deg(colatitude),product_2_4[:, 0, 0],color='green',alpha=0.5)

plt.hlines(y=0, xmin=0, xmax=180, linestyles='--', colors='k')
plt.xlim([0, 180])
plt.legend()

plt.title("Orthogonality of Legendre Polynomials - different degree with same order")
plt.xlabel("Co-latitude")
plt.ylabel("plm(l, m)")

plt.show()

It can be observed that the product of PLM(2,0) and PLM4(4,0) yields a output symmetric about X-axis and also about co-latitude = 90 deg. Thus area under 
the plot turns out to be zero -> both are orthogonal functions

In [ ]:
# the plm implementation using a co-latitude vector (geodetic use-case)

l3 = np.zeros((1,1))
l3[0] = 3

colatitude = np.deg2rad(np.arange(0, 180.5, 0.5))

plm_3_0 = PLM(l=l3, m=0, thetaRAD=colatitude, nargin=1, nargout=1)

product_3_3 = plm_3_0*plm_3_0

# Plotting the plm 
fig, ax = plt.subplots(figsize=(9, 5))
plt.plot(np.rad2deg(colatitude), plm_2_0[:, 0, 0], label="PLM(3, 0)")

plt.plot(np.rad2deg(colatitude), product_3_3[:, 0, 0], label="PLM(3, 0)*PLM(3, 0)", color='green')

plt.fill_between(np.rad2deg(colatitude),product_3_3[:, 0, 0],color='green',alpha=0.5)

plt.hlines(y=0, xmin=0, xmax=180, linestyles='--', colors='k')
plt.xlim([0, 180])
plt.legend()

plt.title("Orthogonality of Legendre Polynomials (same order and same degree)")
plt.xlabel("Co-latitude")
plt.ylabel("plm(l, m)")

plt.show()

In this case the integral cannot be zero, its some finite positive quantity.

In [ ]:
# the plm implementation using a co-latitude vector (geodetic use-case)
l1 = np.zeros((1,1))
l1[0] = 3

l2 = np.zeros((1,1))
l2[0] = 3


colatitude = np.deg2rad(np.arange(0, 180.5, 0.5))

plm_3_1 = PLM(l=l1, m=1, thetaRAD=colatitude, nargin=1, nargout=1)
plm_3_0 = PLM(l=l2, m=2, thetaRAD=colatitude, nargin=1, nargout=1)

product_33_10 = plm_3_1*plm_3_0

# Plotting the plm 
fig, ax = plt.subplots(figsize=(9, 5))
plt.plot(np.rad2deg(colatitude), plm_3_1[:, 0, 0], label="PLM(3, 1)")
plt.plot(np.rad2deg(colatitude), plm_3_0[:, 0, 0], label="PLM(3, 2)")

plt.plot(np.rad2deg(colatitude), product_33_10[:, 0, 0], label="PLM(3, 1)*PLM(3, 2)")

plt.fill_between(np.rad2deg(colatitude),product_33_10[:, 0, 0],color='green',alpha=0.5)

plt.hlines(y=0, xmin=0, xmax=180, linestyles='--', colors='k')
plt.xlim([0, 180])
plt.legend()

plt.title("Orthogonality of Legendre Polynomials - Same degree with different order")
plt.xlabel("Co-latitude")
plt.ylabel("plm(l, m)")

plt.show()

In [ ]:
# the plm implementation using a co-latitude vector (geodetic use-case)
l1 = np.zeros((1,1))
l1[0] = 3

l2 = np.zeros((1,1))
l2[0] = 4


colatitude = np.deg2rad(np.arange(0, 180.5, 0.5))

plm_3_1 = PLM(l=l1, m=1, thetaRAD=colatitude, nargin=1, nargout=1)
plm_3_0 = PLM(l=l2, m=2, thetaRAD=colatitude, nargin=1, nargout=1)

product_33_10 = plm_3_1*plm_3_0

# Plotting the plm 
fig, ax = plt.subplots(figsize=(9, 5))
plt.plot(np.rad2deg(colatitude), plm_3_1[:, 0, 0], label="PLM(3, 1)")
plt.plot(np.rad2deg(colatitude), plm_3_0[:, 0, 0], label="PLM(4, 2)")

plt.plot(np.rad2deg(colatitude), product_33_10[:, 0, 0], label="PLM(3, 1)*PLM(3, 2)")

plt.fill_between(np.rad2deg(colatitude),product_33_10[:, 0, 0],color='green',alpha=0.5)

plt.hlines(y=0, xmin=0, xmax=180, linestyles='--', colors='k')
plt.xlim([0, 180])
plt.legend()

plt.title("Orthogonality of Legendre Polynomials - Different degree with different order")
plt.xlabel("Co-latitude")
plt.ylabel("plm(l, m)")

plt.show()

The shaded regions if symmetric (such that sum=0) indicates that both the polynomials are orthogonal. Else both the functions are not symmetric.

## Surface Spherical Harmonics

similar to the plm and plmplot in SHBundle

$$

$$

In [ ]:
ylmc, ylms = ylm(l=2, m=1)

In [ ]:
ylmc.shape

In [ ]:
np.linspace(0,np.pi,37)

In [ ]:
# basic plotting of SH coeff on plane
l = 0
m = 0

ylmc_00, ylms_00 = ylm(l=l, m=m)

fig = plt.figure(figsize=(15, 7.5))
ax = plt.axes(projection = ccrs.PlateCarree())

lons = np.linspace(-180, 180, 73)
lats = np.linspace(-90, 90, 37)

x, y = np.meshgrid(lons, lats)

if m >=0 :

    img_extent = (-180, 180, -90, 90)

    # plot the data
    im = ax.imshow(ylmc_00[:, 0, :], origin='upper', extent=img_extent, transform=ccrs.PlateCarree(),)
else:
    plt.contourf(x, y, ylms_00[:, 0, :], cmap='RdYlBu_r')


# setting gridlines
gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels=True, x_inline=False, y_inline=False, color='gray', alpha=0.9, linestyle='--')
# remove top x label
gl.top_labels = False
# change x label styles - font size ad colour
gl.xlabel_style = {'size':12,}
# left and right labels
gl.left_labels = True
gl.right_labels = False
# coastlines
ax.coastlines()

plt.colorbar(im, orientation='vertical', shrink=0.85, pad=0.02,label=f"[...]")

plt.title(f"Visualization of Spherical Harmonics - degree: {l} order: {m}")
plt.show()


Instead we can rely on the 'ylm_plot' utility in the visualization module

In [ ]:
# basic plotting of SH coeff on plane - Zonal
ylmc_40, ylms_40 = ylm(l=4, m=0)

ylm_plot(l=4, m=0)

In [ ]:
# basic plotting of SH coeff on plane - Teseral
l = 5
m = 5
ylmc_55, ylms_55 = ylm(l=l, m=m)

ylm_plot(l, m)

In [ ]:
# basic plotting of SH coeff on plane - Sectoral
l = 11
m = 8
ylmc_11_8, ylms_11_8 = ylm(l=l, m=m)

ylm_plot(l, m)

In [ ]:
# Summing up (Superposition) SH Coeff C00 + C10 + S0-1 + C01
# basic plotting of SH coeff on plane - Sectoral

# for explaination purpose
C00 = 1.0
C10 = 10.0
C55 = 15.0
C11_8 = 5.0

ylmc = C00*ylmc_00 + C10*ylmc_40 + C55*ylmc_55 + C11_8*ylmc_11_8


In [ ]:
# Summing SH Coeff C00 + C10 + S0-1 + C01
# basic plotting of SH coeff on plane - Sectoral
l = 2
m = 2

ylmc = C00*ylmc_00 + C10*ylmc_40 + C55*ylmc_55 + C11_8*ylmc_11_8

fig = plt.figure(figsize=(15, 7.5))
ax = plt.axes(projection = ccrs.Orthographic(central_longitude=74.0, central_latitude=0.0, globe=None))

lons = np.linspace(-180, 180, 73)
lats = np.linspace(-90, 90, 37)

# max_level = 6
# min_level = -4.5
# step_level = 0.1
# levels = np.arange(min_level, max_level + step_level, step_level)
x, y = np.meshgrid(lons, lats)

if m >=0 :
    # plt.contourf(x, y, ylmc[:, 0, :], cmap='RdYlBu_r', )
    img_extent = (-180, 180, -90, 90)

    # plot the data
    im = ax.imshow(ylmc[:, 0, :], origin='upper', extent=img_extent, transform=ccrs.PlateCarree(),)
else:
    plt.contourf(x, y, ylms[:, 0, :], cmap='RdYlBu_r')


# setting gridlines
gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels=True, x_inline=False, y_inline=False, color='gray', alpha=0.9, linestyle='--')
# remove top x label
gl.top_labels = False
# change x label styles - font size ad colour
gl.xlabel_style = {'size':12,}
# left and right labels
gl.left_labels = True
gl.right_labels = False
# coastlines
ax.coastlines()

plt.colorbar(im, orientation='vertical', shrink=0.84, pad=0.02,label=f"[...]")

plt.title(f"Surface Spherical Harmonics - Summation - C00 + C40 + C55 + C11_8")
plt.show()


* Thank you for reading